In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Problem 1
Assume that the stock price follows the following process:

$$
dS_t = rS_tdt + \sigma S_tdW_t
$$

Now, consider simulation of a European put with the following parameters: $S_0 = 100$, $K = 110$, $r = 4.75\%$, $\sigma = 20\%$, and maturity $T = 0.5$.

(a) Use the Euler method with the time step size $\Delta t = 0.0005$ to generate 10,000 realizations of $S_T$ and compute the value of the discounted payoff

$$
V_0^{(i)} = e^{-rT}(K - S_T^{(i)})^+
$$

Estimate the mean and the variance of $V_0$. Compare the value of the put obtained using Monte Carlo simulation with the Black–Scholes formula.



(c) Repeat the analysis in (b) using the Milstein method. Is there any significant improvement? If yes, why? If not, why not?

In [45]:
# a

dt = 0.0005
N = 10000
T = 0.5
t = np.arange(0, T+dt/2, dt)
s0 = 100
K = 110
r = 0.0475
sigma = 0.2
s = np.zeros((N, len(t)))
s[:,0] = s0

for i in range(N):
    for k in range(1, len(t)):
        s[i, k] = s[i, k-1] + r * s[i, k-1] * dt + sigma * s[i, k-1] * np.sqrt(dt) * np.random.randn()
V = np.exp(-r*T) * np.maximum(K - s[:, -1], 0)

In [56]:
mu = np.mean(V)
var_v = np.var(V)
print("The mean of V0 is {}".format(mu))
print("The variance of V0 is {}".format(var_v))

The mean of V0 is 10.257595558363597
The variance of V0 is 96.97787974922744


In [50]:
# Black scholes put

from scipy.stats import norm
s0 = 100
K = 110
r = 0.0475
sigma = 0.2
T = 0.5
def d1(s0, k, r, sigma, T):
    nume = np.log(s0/k) + (r + sigma**2 / 2) * T
    den = sigma * np.sqrt(T)
    return nume/den

def d2(d1, sigma, T):
    return d1 - sigma * np.sqrt(T)

def put(s0, k, r, sigma, T):
    d_1 = d1(s0, k, r, sigma, T)
    d_2 = d2(d_1, sigma, T)
    return k * np.exp(-r*T) * norm.cdf(-d_2) - s0 * norm.cdf(-d_1)

v_put = put(s0, K, r, sigma, T)
print("The value of put from black scholes is {}".format(v_put))

The value of put from black scholes is 10.28669994249168


## (b) Repeat the above analysis using the antithetic variates. What can you conclude?

In [53]:
dt = 0.0005
N = 10000
T = 0.5
t = np.arange(0, T+dt/2, dt)
s0 = 100
K = 110
r = 0.0475
sigma = 0.2
s_plus = np.zeros((N, len(t)))
s_minus = np.zeros((N, len(t)))
s_plus[:,0] = s0
s_minus[:,0] = s0

for i in range(N):
    for k in range(1, len(t)):
        s_plus[i, k] = s_plus[i, k-1] + r * s_plus[i, k-1] * dt + sigma * s[i, k-1] * np.sqrt(dt) * np.random.randn()
        s_minus[i, k] = s_minus[i, k-1] + r* s_minus[i, k-1] * dt - sigma * s[i, k-1] * np.sqrt(dt) * np.random.randn()

v = np.exp(-r * T) * np.maximum(K - s_plus[:, -1], 0)
v_anti = np.exp(-r * T) * np.maximum(K - s_minus[:, -1], 0)
call_avg = (v + v_anti)/2

In [62]:
mu = np.mean(call_avg)
var = np.var(call_avg)

print("The mean of the estimator is {}".format(mu))
print("The variance of the estimator is {}".format(var))
print("The variance reduction of we get from this method is {} %".format((var_v - var) / var_v * 100))

The mean of the estimator is 10.08128788868456
The variance of the estimator is 57.88059217737726
The variance reduction of we get from this method is 40.315675773641196 %


We noticed the estimator is roughly the same, but we have variance reduction of 40 percent which is a really good outcome.

# Problem 2

Assume that the stock price follows the following process:

$$
dS_t = rS_tdt + \sigma(S_t, t)S_tdW_t
$$

For the following parameters: spot price $S_0 = 100$, strike price $ K = 110 $, risk-free rate $r = 4.75\% $, local volatility surface $ \sigma(S, t) = 0.5e^{-t(100/S)^{0.3}} $, and maturity $ T = 1.0 $, price a European put option via

(a) finite differences

(b) Monte Carlo simulation

(c) Monte Carlo simulation with a control variate

and compare.